<a href="https://colab.research.google.com/github/ykitaguchi77/FundusPhoto/blob/main/SeqNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Running Github codes (SeqNet) in Google Colab



In [12]:
#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
#Githubよりリポジトリをcloneする
!git clone https://github.com/conscienceli/SeqNet.git

Cloning into 'SeqNet'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 48 (delta 15), reused 41 (delta 10), pack-reused 0
Unpacking objects: 100% (48/48), done.


#作業ディレクトリを移動

In [2]:
import os
import shutil

path = '/content/SeqNet'

#作業ディレクトリをpathに移動する
os.chdir(path)

#作業ディレクトリ直下のファイルを確認
!ls

LICENSE  predict.py  requirements-conda.txt  train.py
pics	 README.md   requirements-pip.txt    utils


In [22]:
#評価画像用のフォルダを作成
data_dir = "./data/test_images/"

if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.makedirs(data_dir)

#copy example data
shutil.copy("/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_img/img00085008_00_1R.jpg", "./data/test_images/image1.jpg")

'./data/test_images/image1.jpg'

In [23]:
#Download pretrained model from Gdrive
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

file_id = '1OYjzu0gixtga6e7Rvb2mZoSSYJkXWRNB'

os.makedirs('./trained_model/ALL', exist_ok=True)
destination = './trained_model/ALL/Final_Emer_Iteration_3_cropsize_128_epochs_200.hdf5'
download_file_from_google_drive(file_id, destination)

In [10]:
!pip uninstall tensorflow
!pip install tensorflow==1.15.0
!pip uninstall keras
!pip3 install keras==2.3.1


      Successfully uninstalled tensorflow-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.13.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.15.0 which is incompatible.
Found existing installation: keras 2.6.0
Uninstalling keras-2.6.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/keras-2.6.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/keras/*
Proceed (y/n)? y
  Successfully uninstalled keras-2.6.0
     |████████████████████████████████| 377 kB 7.9 MB/s 


#**Run codes**

In [19]:
!python predict.py -i ./data/test_images/ -o ./output/

Using TensorFlow backend.



2021-08-30 10:19:06.931605: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-08-30 10:19:06.935730: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000200000 Hz
2021-08-30 10:19:06.936047: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559d9c686a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-08-30 10:19:06.936085: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-08-30 10:19:06.937645: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-08-30 10:19:07.065745: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n